<p style="text-align:center">
    <img alt="Ragas" src="https://github.com/explodinggradients/ragas/blob/main/docs/_static/imgs/logo.png?raw=true" width="250">
<br>
<p>
  <a href="https://github.com/explodinggradients/ragas">Ragas</a>
  |
  <a href="https://docs.arize.com/arize/">Arize</a>
  |
  <a href="https://arize-ai.slack.com/join/shared_invite/zt-11t1vbu4x-xkBIHmOREQnYnYDH1GDfCg">Slack Community</a>
</p>

<center><h1>Tracing and Evaluating AI agents</h1></center>

This guide will walk you through the process of creating and evaluating agents using Ragas and Arize Phoenix. We'll cover the following steps:

* Build an agent that solves math problems with the OpenAI Agents SDK

* Trace agent activity to monitor interactions

* Generate a benchmark dataset for performance analysis

* Evaluate agent performance using Ragas

# Initial setup


We'll setup our libraries, keys, and OpenAI tracing using Phoenix.

### Install Libraries

In [ ]:
!pip install -q "arize-phoenix>=8.0.0" arize-phoenix-otel openinference-instrumentation-openai-agents openinference-instrumentation-openai arize-phoenix-evals "arize[Datasets]" --upgrade

!pip install langchain-openai

!pip install -q openai opentelemetry-sdk opentelemetry-exporter-otlp gcsfs nest_asyncio ragas openai-agents --upgrade

### Setup Keys

Next you need to connect to Phoenix. The code below will connect you to a Phoenix Cloud instance. You can also [connect to a self-hosted Phoenix instance](https://arize.com/docs/phoenix/deployment) if you'd prefer.

In [ ]:
import os
from getpass import getpass

import nest_asyncio

nest_asyncio.apply()

os.environ["PHOENIX_COLLECTOR_ENDPOINT"] = "https://app.phoenix.arize.com"
if not os.environ.get("PHOENIX_CLIENT_HEADERS"):
    os.environ["PHOENIX_CLIENT_HEADERS"] = "api_key=" + getpass("Enter your Phoenix API key: ")

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API key: ")

### Setup Tracing

In [ ]:
from phoenix.otel import register

# Setup OpenTelemetry
tracer_provider = register(
    project_name="agents-cookbook",
    endpoint="https://app.phoenix.arize.com/v1/traces",
    auto_instrument=True,
)

# Create your first agent with the OpenAI SDK

Here we've setup a basic agent that can solve math problems.

We have a function tool that can solve math equations, and an agent that can use this tool.

We'll use the `Runner` class to run the agent and get the final output.

In [ ]:
from agents import Runner, function_tool


@function_tool
def solve_equation(equation: str) -> str:
    """Use python to evaluate the math equation, instead of thinking about it yourself.

    Args:"
       equation: string which to pass into eval() in python
    """
    return str(eval(equation))

In [ ]:
from agents import Agent

agent = Agent(
    name="Math Solver",
    instructions="You solve math problems by evaluating them with python and returning the result",
    tools=[solve_equation],
)

In [ ]:
result = await Runner.run(agent, "what is 15 + 28?")

# Run Result object
print(result)

# Get the final output
print(result.final_output)

# Get the entire list of messages recorded to generate the final output
print(result.to_input_list())

Now we have a basic agent, let's evaluate whether the agent responded correctly.

# Evaluating our agent

Agents can go awry for a variety of reasons. We can use Ragas to evaluate whether the agent responded correctly. Two Ragas measurements help with this:
1. Tool call accuracy - did our agent choose the right tool with the right arguments?
2. Agent goal accuracy - did our agent accomplish the stated goal and get to the right outcome?

Let's setup our evaluation by defining our task function, our evaluator, and our dataset.

In [ ]:
import asyncio

from agents import Runner


# This is our task function. It takes a question and returns the final output and the messages recorded to generate the final output.
async def solve_math_problem(input):
    if isinstance(input, dict):
        input = next(iter(input.values()))
    result = await Runner.run(agent, input)
    return {"final_output": result.final_output, "messages": result.to_input_list()}


result = asyncio.run(solve_math_problem("What is 15 + 28?"))
print(result)

This is helper code which converts the agent messages into a format that Ragas can use.

In [ ]:
def conversation_to_ragas_sample(messages, reference_equation=None, reference_answer=None):
    """
    Convert a single conversation into a Ragas MultiTurnSample.

    Args:
        conversation: Dictionary containing conversation data with 'conversation' key
        reference_equation: Optional string with the reference equation for evaluation

    Returns:
        MultiTurnSample: Formatted sample for Ragas evaluation
    """

    import json

    from ragas.dataset_schema import MultiTurnSample
    from ragas.messages import AIMessage, HumanMessage, ToolCall, ToolMessage

    ragas_messages = []
    pending_tool_call = None
    reference_tool_calls = None

    for item in messages:
        role = item.get("role")
        item_type = item.get("type")

        if role == "user":
            ragas_messages.append(HumanMessage(content=item["content"]))
            pending_tool_call = None

        elif item_type == "function_call":
            args = json.loads(item["arguments"])
            pending_tool_call = ToolCall(name=item["name"], args=args)

        elif item_type == "function_call_output":
            if pending_tool_call is not None:
                ragas_messages.append(AIMessage(content="", tool_calls=[pending_tool_call]))
                ragas_messages.append(
                    ToolMessage(
                        content=item["output"],
                        name=pending_tool_call.name,
                        tool_call_id="tool_call_1",
                    )
                )
                pending_tool_call = None
            else:
                print("[WARN] ToolMessage without preceding ToolCall — skipping.")

        elif role == "assistant":
            content = (
                item["content"][0]["text"]
                if isinstance(item.get("content"), list)
                else item.get("content", "")
            )
            ragas_messages.append(AIMessage(content=content))
    print("Ragas_messages", ragas_messages)

    if reference_equation:
        # Look for the first function call to extract the actual tool call
        for item in messages:
            if item.get("type") == "function_call":
                args = json.loads(item["arguments"])
                reference_tool_calls = [ToolCall(name=item["name"], args=args)]
                break

        return MultiTurnSample(user_input=ragas_messages, reference_tool_calls=reference_tool_calls)

    elif reference_answer:
        return MultiTurnSample(user_input=ragas_messages, reference=reference_answer)

    else:
        return MultiTurnSample(user_input=ragas_messages)

In [ ]:
# Here is an example of the function in action
sample = conversation_to_ragas_sample(
    # This is a list of messages recorded for "Calculate 15 + 28."
    result["messages"],
    reference_equation="15 + 28",
    reference_answer="43",
)
print(sample)

Now let's setup our evaluator. We'll import both metrics we're measuring from Ragas, and use the `multi_turn_ascore(sample)` to get the results.

The `AgentGoalAccuracyWithReference` metric compares the final output to the reference to see if the goal was accomplished.

The `ToolCallAccuracy` metric compares the tool call to the reference tool call to see if the tool call was made correctly.

In [ ]:
from langchain_openai import ChatOpenAI
from ragas.llms import LangchainLLMWrapper
from ragas.metrics import AgentGoalAccuracyWithReference, ToolCallAccuracy


# Setup evaluator LLM and metrics
async def tool_call_evaluator(input, output):
    sample = conversation_to_ragas_sample(output["messages"], reference_equation=input["question"])
    tool_call_accuracy = ToolCallAccuracy()
    return await tool_call_accuracy.multi_turn_ascore(sample)


async def goal_evaluator(input, output):
    sample = conversation_to_ragas_sample(
        output["messages"], reference_answer=output["final_output"]
    )
    evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))
    goal_accuracy = AgentGoalAccuracyWithReference(llm=evaluator_llm)
    return await goal_accuracy.multi_turn_ascore(sample)


print(result)

# Create synthetic dataset of questions

Using the template below, we're going to generate a dataframe of 10 questions we can use to test our math problem solving agent.

In [ ]:
MATH_GEN_TEMPLATE = """
You are an assistant that generates diverse math problems for testing a math solver agent.
The problems should include:

Basic Operations: Simple addition, subtraction, multiplication, division problems.
Complex Arithmetic: Problems with multiple operations and parentheses following order of operations.
Exponents and Roots: Problems involving powers, square roots, and other nth roots.
Percentages: Problems involving calculating percentages of numbers or finding percentage changes.
Fractions: Problems with addition, subtraction, multiplication, or division of fractions.
Algebra: Simple algebraic expressions that can be evaluated with specific values.
Sequences: Finding sums, products, or averages of number sequences.
Word Problems: Converting word problems into mathematical equations.

Do not include any solutions in your generated problems.

Respond with a list, one math problem per line. Do not include any numbering at the beginning of each line.
Generate 10 diverse math problems. Ensure there are no duplicate problems.
"""

In [ ]:
import nest_asyncio
import pandas as pd

from phoenix.evals import OpenAIModel

nest_asyncio.apply()
pd.set_option("display.max_colwidth", 500)

# Initialize the model
model = OpenAIModel(model="gpt-4o", max_tokens=1300)

# Generate math problems
resp = model(MATH_GEN_TEMPLATE)

# Create DataFrame
split_response = resp.strip().split("\n")
math_problems_df = pd.DataFrame(split_response, columns=["question"])
math_problems_df = math_problems_df[math_problems_df["question"].str.strip() != ""]

print(math_problems_df)

Now let's use this dataset and run it with the agent.

In [ ]:
import asyncio

from agents import Runner

# Run agent on all problems and collect conversation history
conversations = []
for idx, row in math_problems_df.iterrows():
    result = asyncio.run(solve_math_problem(row["question"]))
    conversations.append(
        {
            "question": row["question"],
            "final_output": result["final_output"],
            "messages": result["messages"],
        }
    )

print(f"Processed {len(conversations)} problems")
print(conversations[0])

# Create an experiment

With our dataset of questions we generated above, we can use our experiments feature to track changes across models, prompts, parameters for our agent.

Let's create this dataset and upload it into the platform.

In [ ]:
import phoenix as px

dataset_df = pd.DataFrame(
    {
        "question": [conv["question"] for conv in conversations],
        "final_output": [conv["final_output"] for conv in conversations],
    }
)

dataset = px.Client().upload_dataset(
    dataframe=dataset_df,
    dataset_name="math-questions",
    input_keys=["question"],
    output_keys=["final_output"],
)

print(dataset)

Finally, we run our experiment and view the results in Phoenix.

In [ ]:
from phoenix.experiments import run_experiment

experiment = run_experiment(
    dataset, task=solve_math_problem, evaluators=[goal_evaluator, tool_call_evaluator]
)

![Results](https://storage.googleapis.com/arize-phoenix-assets/assets/gifs/ragas_results.gif)